In [1]:
import pyodbc

def test_sql_connection():
    try:
        conn = pyodbc.connect(
            r'DRIVER={ODBC Driver 17 for SQL Server};'
            r'SERVER=localhost\sqlexpress;'
            r'Trusted_Connection=yes;'
        )
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sys.databases")
        print("✅ Connection successful. Databases found:")
        for row in cursor.fetchall():
            print(f" - {row.name}")
        conn.close()
    except Exception as e:
        print("❌ Connection failed:")
        print(e)

if __name__ == "__main__":
    test_sql_connection()


✅ Connection successful. Databases found:
 - master
 - tempdb
 - model
 - msdb


In [12]:
import os
import subprocess
import time
from pathlib import Path
import pyodbc

# === Configuration ===
sql_server_instance = r"BRENDANHALL2D11\SQL_MACPRO_MAUTH"
restore_script_path = Path(r"C:\Temp\ATFS\restore_local_BPH.sql")
backup_dir = Path(r"C:\Temp\ATFS\atfs_bak_20250408_020000")
delete_after = False  # Set to True to auto-delete .bak files after restore

backup_files = [
    backup_dir / "atfs.bak",
    backup_dir / "atfs_log.bak",
    backup_dir / "atfs_gdb.bak",
    backup_dir / "atfs_gdb_log.bak"
]

# === Drop database if exists ===
def drop_db_if_exists(server, db_name):
    try:
        conn = pyodbc.connect(
            rf'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};Trusted_Connection=yes;',
            autocommit=True
        )
        cursor = conn.cursor()

        # Check if DB exists
        cursor.execute("SELECT name FROM sys.databases WHERE LOWER(name) = LOWER(?)", (db_name,))
        if cursor.fetchone():
            print(f"⚠️  Dropping existing database '{db_name}'...")

            # Kill active sessions
            cursor.execute(f"""
                DECLARE @kill varchar(8000) = '';
                SELECT @kill = @kill + 'KILL ' + CONVERT(varchar(5), session_id) + ';'
                FROM sys.dm_exec_sessions
                WHERE database_id = DB_ID('{db_name}') AND session_id <> @@SPID;
                EXEC(@kill);
            """)

            # Drop the database
            cursor.execute(f"ALTER DATABASE [{db_name}] SET SINGLE_USER WITH ROLLBACK IMMEDIATE;")
            cursor.execute(f"DROP DATABASE [{db_name}];")
            print(f"✅ Dropped '{db_name}'.")
        else:
            print(f"ℹ️  Database '{db_name}' does not exist.")
        conn.close()
    except Exception as e:
        print(f"❌ Error dropping database '{db_name}': {e}")

# === Run SQL restore script via sqlcmd ===
def run_sqlcmd(script_path):
    try:
        print("🚀 Restoring databases from script...")
        start = time.time()

        result = subprocess.run([
            "sqlcmd",
            "-S", sql_server_instance,
            "-E",
            "-i", str(script_path),
            "-b"  # stop on first error
        ], capture_output=True, text=True)

        duration = time.time() - start

        print("📤 SQLCMD OUTPUT:")
        print(result.stdout)

        if result.stderr:
            print("❌ SQLCMD ERRORS:")
            print(result.stderr)

        if result.returncode == 0:
            print(f"✅ Restore complete in {duration:.2f} seconds.")
        else:
            print(f"⚠️ sqlcmd exited with return code {result.returncode}. Check above for errors.")

    except subprocess.CalledProcessError as e:
        print("❌ sqlcmd failed:", e)

# === Delete .bak files ===
def delete_files(paths):
    for path in paths:
        try:
            if path.exists():
                path.unlink()
                print(f"🗑️  Deleted: {path}")
            else:
                print(f"📁 Not found (skipped): {path}")
        except Exception as e:
            print(f"⚠️  Could not delete {path}: {e}")

# === Main workflow ===
def main():
    print("🔍 Verifying script and backup files...")

    if not restore_script_path.exists():
        print(f"❌ Restore script not found: {restore_script_path}")
        return

    missing = [f for f in backup_files if not f.exists()]
    if missing:
        print("❌ Missing required backup files:")
        for f in missing:
            print(f" - {f}")
        return

    # Drop if already exists
    drop_db_if_exists(sql_server_instance, "atfs")
    drop_db_if_exists(sql_server_instance, "atfs_gdb")

    # Run restore
    run_sqlcmd(restore_script_path)

    # Optional: delete .bak files
    if delete_after:
        delete_files(backup_files)

    print("✅ All operations complete.")

if __name__ == "__main__":
    main()


🔍 Verifying script and backup files...
⚠️  Dropping existing database 'atfs'...
✅ Dropped 'atfs'.
⚠️  Dropping existing database 'atfs_gdb'...
✅ Dropped 'atfs_gdb'.
🚀 Restoring databases from script...
📤 SQLCMD OUTPUT:
Changed database context to 'master'.
ƒo. Starting restore for [atfs]...
Processed 276496 pages for database 'atfs', file 'atfs' on file 1.
Processed 3 pages for database 'atfs', file 'atfs_log' on file 1.
Converting database 'atfs' from version 869 to the current version 957.
Database 'atfs' running the upgrade step from version 869 to version 875.
Database 'atfs' running the upgrade step from version 875 to version 876.
Database 'atfs' running the upgrade step from version 876 to version 877.
Database 'atfs' running the upgrade step from version 877 to version 878.
Database 'atfs' running the upgrade step from version 878 to version 879.
Database 'atfs' running the upgrade step from version 879 to version 880.
Database 'atfs' running the upgrade step from version 880 t